# The imports needed

In [3]:
import pandas as pd
import sqlite3
from dataclasses import dataclass, fields
from datetime import datetime, timedelta
from typing import Optional

### Looking at the csv data

In [4]:
# Define the paths to the CSV files
csv_files = {
    "circuits": "../data/circuits.csv",
    "constructor_results": "../data/constructor_results.csv",
    "constructor_standings": "../data/constructor_standings.csv",
    "constructors": "../data/constructors.csv",
    "driver_standings": "../data/driver_standings.csv",
    "drivers": "../data/drivers.csv",
    "lap_times": "../data/lap_times.csv",
    "pit_stops": "../data/pit_stops.csv",
    "qualifying": "../mnt/data/qualifying.csv",
    "races": "../data/races.csv",
    "results": "../data/results.csv",
    "seasons": "../data/seasons.csv",
    "sprint_results": "../data/sprint_results.csv",
    "status": "../data/status.csv"
}

# Read the CSV files and store their structure (first few rows)
csv_structures = {}

for file_name, file_path in csv_files.items():
    try:
        df = pd.read_csv(file_path)
        csv_structures[file_name] = df.head()
    except Exception as e:
        csv_structures[file_name] = f"Error reading file: {e}"

csv_structures

{'circuits':    circuitId   circuitRef                            name      location  \
 0          1  albert_park  Albert Park Grand Prix Circuit     Melbourne   
 1          2       sepang    Sepang International Circuit  Kuala Lumpur   
 2          3      bahrain   Bahrain International Circuit        Sakhir   
 3          4    catalunya  Circuit de Barcelona-Catalunya      Montmeló   
 4          5     istanbul                   Istanbul Park      Istanbul   
 
      country       lat        lng  alt  \
 0  Australia -37.84970  144.96800   10   
 1   Malaysia   2.76083  101.73800   18   
 2    Bahrain  26.03250   50.51060    7   
 3      Spain  41.57000    2.26111  109   
 4     Turkey  40.95170   29.40500  130   
 
                                                  url  
 0  http://en.wikipedia.org/wiki/Melbourne_Grand_P...  
 1  http://en.wikipedia.org/wiki/Sepang_Internatio...  
 2  http://en.wikipedia.org/wiki/Bahrain_Internati...  
 3  http://en.wikipedia.org/wiki/Circuit_de_Ba

# Dataclass

In [5]:
@dataclass
class Circuit:
    circuitId: int  # Primary Key, auto ID int
    circuitRef: str  # Short name for the circuit
    name: str  # Long version of the name
    location: str  # City name
    country: str  # Country it's in
    lat: float  # Latitude up to 4 decimals
    lng: float  # Longitude up to 4 decimals
    alt: int  # Altitude of the circuit (integer, handle \N separately)
    url: str  # HTTPS URL of the Wikipedia page

    def __post_init__(self):
        for field in fields(self):
            value = getattr(self, field.name)
            if value == '\\N':
                setattr(self, field.name, None)

@dataclass
class Constructor:
    constructorId: int  # Primary Key
    constructorRef: str  # Short name
    name: str  # Full name
    nationality: str  # Full nation name
    url: str  # HTTPS URL of the Wikipedia page

    def __post_init__(self):
        for field in fields(self):
            value = getattr(self, field.name)
            if value == '\\N':
                setattr(self, field.name, None)

@dataclass
class Driver:
    driverId: int  # Primary Key, driver ID
    driverRef: str  # Short name
    number: int  # Driver's number, only for recent drivers
    code: str  # Code on the ticker, e.g., HAM, VER
    forename: str  # First name
    surname: str  # Last name
    dob: datetime  # Date of birth in M/DD/YYYY format
    nationality: str  # Full country name
    url: str  # HTTPS URL of the Wikipedia page

    def __post_init__(self):
        for field in fields(self):
            value = getattr(self, field.name)
            if field.name == 'dob' and isinstance(value, str):
                self.dob = datetime.strptime(value, '%m/%d/%Y') if value != '\\N' else None
            elif value == '\\N':
                setattr(self, field.name, None)

@dataclass
class Season:
    year: int  # Year of the season
    url: str  # HTTPS URL of the season's Wikipedia page

@dataclass
class Status:
    statusId: int  # Primary Key, auto-generated ID
    status: str  # Status description (e.g., Finished, Crashed, etc.)

@dataclass
class Race:
    raceId: int  # Primary Key, auto-generated ID
    year: int  # Year of the race in YYYY format
    round: int  # Round in a year
    circuitId: int  # Foreign key from circuits
    name: str  # Name of the Grand Prix
    date: datetime  # Race date in MM/DD/YYYY format
    time: datetime  # Race time in military time HH:MM:SS
    url: str  # HTTPS URL of the race
    fp1_date: Optional[datetime]  # Free Practice 1 date
    fp1_time: Optional[datetime]  # Free Practice 1 time
    fp2_date: Optional[datetime]  # Free Practice 2 date
    fp2_time: Optional[datetime]  # Free Practice 2 time
    fp3_date: Optional[datetime]  # Free Practice 3 date
    fp3_time: Optional[datetime]  # Free Practice 3 time
    quali_date: Optional[datetime]  # Qualifying date
    quali_time: Optional[datetime]  # Qualifying time
    sprint_date: Optional[datetime]  # Sprint race date
    sprint_time: Optional[datetime]  # Sprint race time

    def __post_init__(self):
        date_time_fields = ['date', 'time', 'fp1_date', 'fp1_time', 'fp2_date', 
                            'fp2_time', 'fp3_date', 'fp3_time', 'quali_date', 
                            'quali_time', 'sprint_date', 'sprint_time']

        for field in fields(self):
            value = getattr(self, field.name)
            if field.name in date_time_fields and isinstance(value, str):
                date_format = '%m/%d/%Y' if 'date' in field.name else '%H:%M:%S'
                setattr(self, field.name, datetime.strptime(value, date_format) if value != '\\N' else None)
            elif value == '\\N':
                setattr(self, field.name, None)

@dataclass
class ConstructorResults:
    constructorResultsId: int  # Primary Key, auto-generated ID
    raceId: int  # Foreign key from Races
    constructorId: int  # Foreign key from Constructors
    points: float  # Total points from the race
    status: Optional[str]  # Status of the constructor's result, handles '\N'

    def __post_init__(self):
        for field in fields(self):
            value = getattr(self, field.name)
            if value == '\\N':
                setattr(self, field.name, None)

@dataclass
class ConstructorStandings:
    constructorStandingsId: int  # Primary Key, auto-generated ID
    raceId: int  # Foreign key from Races
    constructorId: int  # Foreign key from Constructors
    points: float  # Total points from the race
    position: int  # Current position in the points tally
    positionText: str  # Position as a string
    wins: int  # Number of wins in the season up to that point

    def __post_init__(self):
        for field in fields(self):
            value = getattr(self, field.name)
            if value == '\\N':
                setattr(self, field.name, None)

@dataclass
class DriverStandings:
    driverStandingsId: int  # Primary Key, auto-generated ID
    raceId: int  # Foreign key from Races
    driverId: int  # Foreign key from Drivers
    points: float  # Points per driver at that point in the season
    position: int  # Driver's finishing position
    positionText: str  # Finishing position as a string
    wins: int  # Number of wins up to that point in the season

    def __post_init__(self):
        for field in fields(self):
            value = getattr(self, field.name)
            if value == '\\N':
                setattr(self, field.name, None)

@dataclass
class LapTimes:
    raceId: int  # Foreign key from Races
    driverId: int  # Foreign key from Drivers
    lap: int  # Number of the lap
    position: int  # Position in the race of the driver
    time: timedelta  # Lap time in MM:SS.M format
    milliseconds: int  # Lap time in total milliseconds

    def __post_init__(self):
        for field in fields(self):
            value = getattr(self, field.name)
            if field.name == 'time' and isinstance(value, str):
                # Splitting the time into minutes, seconds, and fractions of a second
                minutes, rest = value.split(':')
                seconds, millis = rest.split('.')
                total_seconds = int(minutes) * 60 + int(seconds) + float(f'.{millis}')
                self.time = timedelta(seconds=total_seconds)
            elif value == '\\N':
                setattr(self, field.name, None)

@dataclass              
class PitStops:
    raceId: int  # Foreign key from Races
    driverId: int  # Foreign key from Drivers
    stop: int  # Number of the stop for a driver
    lap: int  # Lap in the race
    time: timedelta  # Time in the race when the stop has taken, MM:SS.MM
    duration: timedelta  # Duration of the pitstop, SS:MMM
    milliseconds: int  # Total milliseconds of the pitstop

    def __post_init__(self):
        for field in fields(self):
            value = getattr(self, field.name)
            if field.name in ['time', 'duration'] and isinstance(value, str):
                # Converting time and duration to timedelta
                if ':' in value:  # Handling MM:SS.MM or SS:MMM format
                    minutes, rest = (value.split(':') + ['0'])[:2]  # Adds '0' minutes if they are missing
                    seconds, millis = rest.split('.')
                    total_seconds = int(minutes) * 60 + int(seconds) + float(f'.{millis}')
                    setattr(self, field.name, timedelta(seconds=total_seconds))
                else:  # Directly converting milliseconds to timedelta
                    setattr(self, field.name, timedelta(milliseconds=int(value)))
            elif value == '\\N':
                setattr(self, field.name, None)

@dataclass
class RaceResults:
    resultId: int  # Primary Key, auto-generated ID
    raceId: int  # Foreign key from Races
    driverId: int  # Foreign key from Drivers
    constructorId: int  # Foreign key from Constructors
    number: int  # The number of the car
    grid: int  # Starting position of the car
    position: Optional[int]  # Finishing position (integer or None)
    positionText: str  # Finishing position in string or R or D
    positionOrder: int  # Complete order with no '\N'
    points: float  # Total points for the car
    laps: int  # Total laps completed
    time: Optional[timedelta]  # Time in MM:SS.M or seconds SS.MMM or +M:SS.MMM
    milliseconds: Optional[int]  # Total milliseconds of the race for each driver
    fastestLap: Optional[int]  # Number of the lap with the fastest lap time
    rank: Optional[int]  # Rank of the fastest lap time
    fastestLapTime: Optional[timedelta]  # Time of the fastest lap MM:SS.M
    fastestLapSpeed: Optional[float]  # Fastest speed achieved in a lap, up to 3 decimals
    statusId: int  # Foreign key from Status

    def __post_init__(self):
        for field in fields(self):
            value = getattr(self, field.name)
            if field.name in ['time', 'fastestLapTime'] and isinstance(value, str):
                if '+' in value:
                    value = value.strip('+')
                if ':' in value:
                    minutes, rest = (value.split(':') + ['0'])[:2]
                    seconds, millis = rest.split('.')
                    total_seconds = int(minutes) * 60 + int(seconds) + float(f'.{millis}')
                    setattr(self, field.name, timedelta(seconds=total_seconds))
                else:
                    setattr(self, field.name, timedelta(milliseconds=int(value)))
            elif isinstance(value, str) and value == '\\N':
                setattr(self, field.name, None)

@dataclass
class SprintResults:
    resultId: int  # Primary Key, auto-generated ID
    raceId: int  # Foreign key from Races
    driverId: int  # Foreign key from Drivers
    constructorId: int  # Foreign key from Constructors
    number: int  # The number of the car
    grid: int  # Starting position of the car
    position: Optional[int]  # Finishing position (integer or None)
    positionText: str  # Finishing position in string or R or D
    positionOrder: int  # Complete order with no '\N'
    points: float  # Total points for the car
    laps: int  # Total laps completed
    time: Optional[timedelta]  # Time in MM:SS.M or seconds SS.MMM or +M:SS.MMM
    milliseconds: Optional[int]  # Total milliseconds of the race for each driver
    fastestLap: Optional[int]  # Number of the lap with the fastest lap time
    fastestLapTime: Optional[timedelta]  # Time of the fastest lap MM:SS.M
    statusId: int  # Foreign key from Status

    def __post_init__(self):
        for field in fields(self):
            value = getattr(self, field.name)
            if field.name in ['time', 'fastestLapTime'] and isinstance(value, str):
                if '+' in value:
                    value = value.strip('+')
                if ':' in value:
                    minutes, rest = (value.split(':') + ['0'])[:2]
                    seconds, millis = rest.split('.')
                    total_seconds = int(minutes) * 60 + int(seconds) + float(f'.{millis}')
                    setattr(self, field.name, timedelta(seconds=total_seconds))
                else:
                    setattr(self, field.name, timedelta(milliseconds=int(value)))
            elif isinstance(value, str) and value == '\\N':
                setattr(self, field.name, None)

# Create the tables in SQLite3.

In [6]:
import sqlite3

# Connect to SQLite database (or create if it doesn't exist)
conn = sqlite3.connect('racing_data.db')
cursor = conn.cursor()

# Creating the tables
# Circuits table 
cursor.execute('''
CREATE TABLE Circuits (
    circuitId INTEGER PRIMARY KEY,
    circuitRef TEXT,
    name TEXT,
    location TEXT,
    country TEXT,
    lat REAL,
    lng REAL,
    alt INTEGER,
    url TEXT
)
''')

# Constructors table 
cursor.execute('''
CREATE TABLE Constructors (
    constructorId INTEGER PRIMARY KEY,
    constructorRef TEXT,
    name TEXT,
    nationality TEXT,
    url TEXT
)
''')

# Drivers table 
cursor.execute('''
CREATE TABLE Drivers (
    driverId INTEGER PRIMARY KEY,
    driverRef TEXT,
    number INTEGER,
    code TEXT,
    forename TEXT,
    surname TEXT,
    dob TEXT,  -- Stored as text in SQLite, converted to datetime in Python
    nationality TEXT,
    url TEXT
)
''')

#Seasons table 
cursor.execute('''
CREATE TABLE Seasons (
    year INTEGER PRIMARY KEY,
    url TEXT
)
''')

# Status table 
cursor.execute('''
CREATE TABLE Status (
    statusId INTEGER PRIMARY KEY,
    status TEXT
)
''')

# Races table
cursor.execute('''
CREATE TABLE Races (
    raceId INTEGER PRIMARY KEY,
    year INTEGER,
    round INTEGER,
    circuitId INTEGER,
    name TEXT,
    date TEXT,  -- Stored as text in SQLite, converted to datetime in Python
    time TEXT,  -- Stored as text in SQLite, converted to datetime in Python
    url TEXT,
    fp1_date TEXT,
    fp1_time TEXT,
    fp2_date TEXT,
    fp2_time TEXT,
    fp3_date TEXT,
    fp3_time TEXT,
    quali_date TEXT,
    quali_time TEXT,
    sprint_date TEXT,
    sprint_time TEXT,
    FOREIGN KEY(circuitId) REFERENCES Circuits(circuitId)
)
''')

# ConstructorResults table
cursor.execute('''
CREATE TABLE ConstructorResults (
    constructorResultsId INTEGER PRIMARY KEY,
    raceId INTEGER,
    constructorId INTEGER,
    points REAL,
    status TEXT,
    FOREIGN KEY(raceId) REFERENCES Races(raceId),
    FOREIGN KEY(constructorId) REFERENCES Constructors(constructorId)
)
''')

# ConstructorStandings table
cursor.execute('''
CREATE TABLE ConstructorStandings (
    constructorStandingsId INTEGER PRIMARY KEY,
    raceId INTEGER,
    constructorId INTEGER,
    points REAL,
    position INTEGER,
    positionText TEXT,
    wins INTEGER,
    FOREIGN KEY(raceId) REFERENCES Races(raceId),
    FOREIGN KEY(constructorId) REFERENCES Constructors(constructorId)
)
''')

# DriverStandings table
cursor.execute('''
CREATE TABLE DriverStandings (
    driverStandingsId INTEGER PRIMARY KEY,
    raceId INTEGER,
    driverId INTEGER,
    points REAL,
    position INTEGER,
    positionText TEXT,
    wins INTEGER,
    FOREIGN KEY(raceId) REFERENCES Races(raceId),
    FOREIGN KEY(driverId) REFERENCES Drivers(driverId)
)
''')

# LapTimes table
cursor.execute('''
CREATE TABLE LapTimes (
    raceId INTEGER,
    driverId INTEGER,
    lap INTEGER,
    position INTEGER,
    time TEXT,  -- Stored as text in SQLite, converted to timedelta in Python
    milliseconds INTEGER,
    FOREIGN KEY(raceId) REFERENCES Races(raceId),
    FOREIGN KEY(driverId) REFERENCES Drivers(driverId)
)
''')

# PitStops table
cursor.execute('''
CREATE TABLE PitStops (
    raceId INTEGER,
    driverId INTEGER,
    stop INTEGER,
    lap INTEGER,
    time TEXT,  -- Stored as text in SQLite, converted to timedelta in Python
    duration TEXT,  -- Stored as text in SQLite, converted to timedelta in Python
    milliseconds INTEGER,
    FOREIGN KEY(raceId) REFERENCES Races(raceId),
    FOREIGN KEY(driverId) REFERENCES Drivers(driverId)
)
''')

# RaceResults table
cursor.execute('''
CREATE TABLE RaceResults (
    resultId INTEGER PRIMARY KEY,
    raceId INTEGER,
    driverId INTEGER,
    constructorId INTEGER,
    number INTEGER,
    grid INTEGER,
    position INTEGER,
    positionText TEXT,
    positionOrder INTEGER,
    points REAL,
    laps INTEGER,
    time TEXT,  -- Stored as text in SQLite, converted to timedelta in Python
    milliseconds INTEGER,
    fastestLap INTEGER,
    rank INTEGER,
    fastestLapTime TEXT,  -- Stored as text in SQLite, converted to timedelta in Python
    fastestLapSpeed REAL,
    statusId INTEGER,
    FOREIGN KEY(raceId) REFERENCES Races(raceId),
    FOREIGN KEY(driverId) REFERENCES Drivers(driverId),
    FOREIGN KEY(constructorId) REFERENCES Constructors(constructorId),
    FOREIGN KEY(statusId) REFERENCES Status(statusId)
)
''')

# SprintResults table
cursor.execute('''
CREATE TABLE SprintResults (
    resultId INTEGER PRIMARY KEY,
    raceId INTEGER,
    driverId INTEGER,
    constructorId INTEGER,
    number INTEGER,
    grid INTEGER,
    position INTEGER,
    positionText TEXT,
    positionOrder INTEGER,
    points REAL,
    laps INTEGER,
    time TEXT,  -- Stored as text in SQLite, converted to timedelta in Python
    milliseconds INTEGER,
    fastestLap INTEGER,
    fastestLapTime TEXT,  -- Stored as text in SQLite, converted to timedelta in Python
    statusId INTEGER,
    FOREIGN KEY(raceId) REFERENCES Races(raceId),
    FOREIGN KEY(driverId) REFERENCES Drivers(driverId),
    FOREIGN KEY(constructorId) REFERENCES Constructors(constructorId),
    FOREIGN KEY(statusId) REFERENCES Status(statusId)
)
''')

# Commit the changes and close the connection
conn.commit()
conn.close()

OperationalError: table Circuits already exists

# Load in the Data

In [8]:
# Define the path to the CSV files
data_folder = '../data/'

# Dictionary mapping class names to CSV file names
csv_files = {
    'Circuits': 'circuits.csv',
    'Constructors': 'constructors.csv',
    'Drivers': 'drivers.csv',
    'Seasons': 'seasons.csv',
    'Status': 'status.csv',
    'Races': 'races.csv',
    'ConstructorResults': 'constructor_results.csv',
    'ConstructorStandings': 'constructor_standings.csv',
    'DriverStandings': 'driver_standings.csv',
    'LapTimes': 'lap_times.csv',
    'PitStops': 'pit_stops.csv',
    'RaceResults': 'results.csv',
    'SprintResults': 'sprint_results.csv'
}

# Connect to SQLite database
conn = sqlite3.connect('racing_data.db')

for table, csv_file in csv_files.items():
    # Construct the full path to the CSV file
    full_path = f'{data_folder}/{csv_file}'

    # Read the CSV file into a DataFrame
    df = pd.read_csv(full_path)

    # Convert the DataFrame to SQL
    df.to_sql(table, conn, if_exists='replace', index=False)

# Close the connection
conn.close()


In [5]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('racing_data.db')
cursor = conn.cursor()

# Execute the query
query = """
SELECT r.year, d.surname, d.forename, ds.points as total_points
FROM DriverStandings ds
JOIN Races r ON ds.raceId = r.raceId
JOIN Drivers d ON ds.driverId = d.driverId
WHERE d.driverId = 30 AND r.round = (SELECT MAX(round) FROM Races WHERE year = r.year)
ORDER BY r.year;
"""
cursor.execute(query)

# Fetch and print the results
results = cursor.fetchall()
for row in results:
    print(row)

# Close the connection
conn.close()


(1991, 'Schumacher', 'Michael', 4.0)
(1992, 'Schumacher', 'Michael', 53.0)
(1993, 'Schumacher', 'Michael', 52.0)
(1994, 'Schumacher', 'Michael', 92.0)
(1995, 'Schumacher', 'Michael', 102.0)
(1996, 'Schumacher', 'Michael', 59.0)
(1997, 'Schumacher', 'Michael', 78.0)
(1998, 'Schumacher', 'Michael', 86.0)
(1999, 'Schumacher', 'Michael', 44.0)
(2000, 'Schumacher', 'Michael', 108.0)
(2001, 'Schumacher', 'Michael', 123.0)
(2002, 'Schumacher', 'Michael', 144.0)
(2003, 'Schumacher', 'Michael', 93.0)
(2004, 'Schumacher', 'Michael', 148.0)
(2005, 'Schumacher', 'Michael', 62.0)
(2006, 'Schumacher', 'Michael', 121.0)
(2010, 'Schumacher', 'Michael', 72.0)
(2011, 'Schumacher', 'Michael', 76.0)
(2012, 'Schumacher', 'Michael', 49.0)
